In [18]:
# 1. 구글드라이브와 colab 연동  
from google.colab import drive  # 구글드라이브와 연동하기 위한 라이브러리
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
# 2. 라이브러리 import
# '''
# pytorch, torchvision, matplotlib, numpy, tqdm, time 라이브러리가 요구된다.
# '''

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

In [20]:
# 3. 하이퍼 파라미터 설정
EPOCH = 150
batch_size = 128
learning_rate = .1
momentum = 0.9

In [21]:
# 4. torch에서 사용할 gpu 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cuda:0 is available


In [22]:
# 5. Classification할 Class list 지정
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [23]:
# 6. 이미지 전처리
transform_train = transforms.Compose([   
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
    )

In [24]:
# 7. (수정불필요) Dataset 저장 및 Tensor 형태로 변환

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
from torch.utils.data import random_split

In [26]:
# 모의 test 시 train_ration에 0.99 할당 / test 시 valid 필요 x-> train_ratio 1
train_ratio = 1
train_size = int(len(trainset) * train_ratio)
valid_size = len(trainset) - train_size
trainset, validset = random_split(trainset, [train_size, valid_size])
print(len(trainset), len(validset))

50000 0


In [27]:
# 모의 test시 validloader #제거
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
# validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)
       
        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out) # 출력: [batch_size, 512, 4, 4]
        out = F.avg_pool2d(out, 4) # 출력: [batch_size, 512, 1, 1]
        out = out.view(out.size(0), -1) # 출력: [batch_size, 512]
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [29]:
# 9. Network 선언 및 device 위에 올리기
from torchsummary import summary
net = ResNet18().to(device)
summary(net, (3, 32, 32), batch_size=batch_size)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [128, 64, 32, 32]           1,728
       BatchNorm2d-2          [128, 64, 32, 32]             128
            Conv2d-3          [128, 64, 32, 32]          36,864
       BatchNorm2d-4          [128, 64, 32, 32]             128
            Conv2d-5          [128, 64, 32, 32]          36,864
       BatchNorm2d-6          [128, 64, 32, 32]             128
        BasicBlock-7          [128, 64, 32, 32]               0
            Conv2d-8          [128, 64, 32, 32]          36,864
       BatchNorm2d-9          [128, 64, 32, 32]             128
           Conv2d-10          [128, 64, 32, 32]          36,864
      BatchNorm2d-11          [128, 64, 32, 32]             128
       BasicBlock-12          [128, 64, 32, 32]               0
           Conv2d-13         [128, 128, 16, 16]          73,728
      BatchNorm2d-14         [128, 128,

In [30]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr /= 10
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [31]:
def evaluate(model, dataloader, criterion):
    with torch.no_grad():
        model.eval()
        n_total, n_correct = 0, 0
        running_loss = 0.0
        for img, labels in dataloader:
            out = model(img.to(device))
            _, y_pred = torch.max(out.data, 1)
            n_correct += (y_pred==labels.to(device)).sum().item()
            n_total += img.size(0)
            running_loss += criterion(out, labels.to(device)).item()
        val_acc = n_correct/n_total
    return val_acc, running_loss/len(dataloader)


In [32]:
import numpy as np

mixup_alpha = 1.0

def mixup_data(x, y):
    lam = np.random.beta(mixup_alpha, mixup_alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, y, targets, smoothing=0.1):
        confidence = 1. - smoothing
        log_probs = F.log_softmax(y, dim=-1) # 예측 확률 계산
        true_probs = torch.zeros_like(log_probs)
        true_probs.fill_(smoothing / (y.shape[1] - 1))
        # 정답 인덱스의 정답 확률을 confidence로 변경
        true_probs.scatter_(1, targets.data.unsqueeze(1), confidence)
        # negative log likelihood 
        return torch.mean(torch.sum(true_probs * -log_probs, dim=-1)) 

In [33]:
net = ResNet18()
net = net.to(device)

# 이전 기록에서 이어서 학습을 원하면 주석을 해제할 것.
#net.load_state_dict(torch.load('/usr/local/lib/python3.10/dist-packages/google/colab/drive.py'))

learning_rate = 0.1

criterion = LabelSmoothingCrossEntropy()
optimizer = optim.SGD(net.parameters(), 
            lr=learning_rate, momentum=0.9, weight_decay=0.0002)


In [ ]:
# 12. Training

train_losses = []
valid_losses = []
accuracies = []

n = len(trainloader)

for epoch in range(EPOCH): 
  adjust_learning_rate(optimizer, epoch)
  running_loss = 0.0
  start = time.time()


  for data in tqdm(trainloader):

    inputs, targets = data[0].to(device), data[1].to(device) # 배치 데이터
    inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
    
    optimizer.zero_grad() # 배치마다 optimizer 초기화
    outputs = net(inputs)
    
    loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam) 
    # Trainging-크로스 엔트로피 손실함수 계산 

    loss.backward() # backpropagation
    # import pdb; pdb.set_trace()
    optimizer.step() # 가중치 최적화

    running_loss += loss.item()

############ valid: 모의 test시 실행
#   valid_acc, valid_loss = evaluate(net, validloader, criterion)
  train_loss = running_loss / n
  train_losses.append(train_loss)
#   valid_losses.append(valid_loss)
#   accuracies.append(valid_acc)
  torch.save(net.state_dict() ,'/usr/local/lib/python3.10/dist-packages/google/colab/drive.py')

  #수정# loss_.append(running_loss / n)
  #수정# print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(trainloader)))
  print('[%d] loss: %.3f' %(epoch + 1, running_loss / n))
  print("epoch time :", time.time()-start)
#   print("valid accuracy : ", round다음(accuracies[-1], 3))
  

 43%|████▎     | 168/391 [00:24<00:34,  6.47it/s]

In [ ]:
#valid <-> train loss 출력
plt.figure(figsize=(14,12))   
plt.title("train vs valid loss")
plt.plot(train_losses, label="TRAIN")
plt.plot(valid_losses, label="VALID")
plt.legend()
plt.show()

In [ ]:
#valid <-> train accuracy 출력
plt.figure(figsize=(14,12))    
plt.title("train vs valid accuracy")
plt.plot(accuracies, label="VALID")
plt.legend()
plt.show()

In [ ]:
# Evaluation
net = ResNet18()
net.to(device)
# !!주의!! weight 저장경로는 상황에 맞게 수정
net.load_state_dict(torch.load('/usr/local/lib/python3.10/dist-packages/google/colab/drive.py'))

# (수정불가) Evaluation
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}  #dictionary 생성
total_pred = {classname: 0 for classname in classes}    #dictionary 생성

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad(): # autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높일 수 있다.
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) # labels.size = batch_size
        correct += (predicted == labels).sum().item()
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

In [ ]:
#Inference
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(testloader)
#수정# images, labels = dataiter.next()
images, labels = next(dataiter)

# GroundTruth display
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(8)))

# Predicted display
net = ResNet18()
net.to(device)
# !!주의!! weight 저장경로는 상황에 맞게 수정
net.load_state_dict(torch.load('/usr/local/lib/python3.10/dist-packages/google/colab/drive.py'))

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(8)))